https://trello.com/c/DTm3W660/10-air-plane-crashes-kaggle

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

plt.style.use('seaborn')

In [2]:
df = pd.read_csv('data.csv')

**1.1: remove columns with 80 or more percent of Null-values**

In [3]:
# 80% of 13 cols je 10
df.dropna(thresh=10, inplace=True)

**1.2: check the ground column to see if it only contains 0 and 1. If so convert it to a Boolean type**

In [4]:
'It contains more, namely {} different values.'.format(df.Ground.nunique())

'It contains more, namely 44 different values.'

**2.1: on avg. how many plane crashes happen per year?**

In [5]:
# Create a year column
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year

In [6]:
crashes_per_year = df['Year'].count() / df['Year'].nunique()
'There are {} crashes per year on average'.format(round(crashes_per_year, 2))

'There are 49.51 crashes per year on average'

**2.1.1: which year was the one with the least/most crashes?**

In [7]:
'The year with the most crashes was {} and the one with the least {}.'.format(df.Year.value_counts().idxmax(), df.Year.value_counts().idxmin())

'The year with the most crashes was 1972 and the one with the least 1923.'

**2.2: on avg. how many months are in between crashes?**

In [8]:
time_between_crashes = (365 / crashes_per_year) / 30
'The average time between crashes is {} months.'.format(round(time_between_crashes,2))

'The average time between crashes is 0.25 months.'

**2.2.1: what was the longest period in between crashes?**

In [9]:
# The dates were not sorted, first we need to do that to be able to calculate
df.sort_values(by='Date', inplace=True)

In [10]:
# Iterate over the dates substracting one of the other
periods = []
nums = range(len(df['Date'])-1)

for num in nums:
    n1 = df.index[num]
    n2 = df.index[num+1]
    y = df['Date'][n2]
    x = df['Date'][n1]
    z = y - x
    periods.append(z)

In [34]:
# 3 largest periods
import heapq

heapq.nlargest(3, periods)

[Timedelta('3007 days 00:00:00'),
 Timedelta('1394 days 00:00:00'),
 Timedelta('625 days 00:00:00')]

**2.3: which Operator suffered the most crashes in history?**

In [12]:
df.Operator.value_counts().nlargest(3)

Military - U.S. Air Force    162
Aeroflot                     150
Air France                    59
Name: Operator, dtype: int64

Overall it was the US Air Force, from comercial airlines it was Aeroflot

**3.1: from the Route column extract start and destination and create new columns for each**

In [13]:
df[['Start', 'Destination']] = df.Route.str.split('-', n=1, expand=True)
df.tail()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Year,Start,Destination
5262,2009-05-03,12:00,"Near El Alto de Rubio, Venezuela",Military - Venezuelan Army,NaN,Patrol,Mi-35,EV08114,NaN,18.0,18.0,0.0,The helicopter was patrolling along the Venezu...,2009,Patrol,None
5263,2009-05-20,06:30,"Near Madiun, Indonesia",Military - Indonesian Air Force,NaN,Jakarta - Maduin,Lockheed C-130 Hercules,A-1325,1982,112.0,98.0,2.0,"While on approach, the military transport cras...",2009,Jakarta,Maduin
5264,2009-05-26,NaN,"Near Isiro, DemocratiRepubliCongo",Service Air,NaN,Goma - Isiro,Antonov An-26,9Q-CSA,5005,4.0,4.0,NaN,The cargo plane crashed while on approach to I...,2009,Goma,Isiro
5265,2009-06-01,00:15,"AtlantiOcean, 570 miles northeast of Natal, Br...",Air France,447,Rio de Janeiro - Paris,Airbus A330-203,F-GZCP,660,228.0,228.0,0.0,The Airbus went missing over the AtlantiOcean ...,2009,Rio de Janeiro,Paris
5266,2009-06-07,08:30,"Near Port Hope Simpson, Newfoundland, Canada",Strait Air,NaN,Lourdes de BlanSablon - Port Hope Simpson,Britten-Norman BN-2A-27 Islander,C-FJJR,424,1.0,1.0,0.0,The air ambulance crashed into hills while att...,2009,Lourdes de BlanSablon,Port Hope Simpson


**3.2: what´s the most common weekday for crashes?**

In [14]:
# Creating a new column
df['Day_Of_Week'] = df.Date.dt.dayofweek

In [15]:
# Counting
df.Day_Of_Week.value_counts()

3    715
2    695
1    686
4    685
5    647
0    606
6    521
Name: Day_Of_Week, dtype: int64

In [16]:
'The weekday with most crashes is Wednesday.'

'The weekday with most crashes is Wednesday.'

**4.1: on avg. how many passengers survive a crash (if any)? Create a new column called survival rate**

In [17]:
df['Survival_Rate'] = 100 - ((df.Fatalities * 100) / df.Aboard)

In [18]:
'The survival rate is {}%.'.format(round(df.Survival_Rate.mean(), 2))

'The survival rate is 17.7%.'

**What were the most common causes for crashes? Could be extraced from the `Summary`column (e.g. most common words)**

In [30]:
# Create new DF
df2 = df.dropna(subset=['Summary']).copy()

In [32]:
# Split the words
summary = df2.Summary.str.split()

In [33]:
# Create a list of those words
flattened = []
for x in summary:
    for y in x:
        flattened.append(y)

In [50]:
# Count the 100 most common ones
from collections import Counter

data = Counter(flattened)
data.most_common(100)

[('the', 11242),
 ('of', 4671),
 ('to', 4670),
 ('and', 4651),
 ('a', 4390),
 ('The', 4072),
 ('in', 3009),
 ('into', 2209),
 ('was', 2189),
 ('aircraft', 2064),
 ('crashed', 1660),
 ('plane', 1621),
 ('on', 1437),
 ('after', 1432),
 ('an', 1257),
 ('at', 1183),
 ('from', 1171),
 ('while', 1152),
 ('Crashed', 1075),
 ('by', 836),
 ('flight', 831),
 ('with', 780),
 ('pilot', 713),
 ('engine', 700),
 ('off', 693),
 ('during', 670),
 ('for', 620),
 ('approach', 607),
 ('crew', 600),
 ('not', 566),
 ('which', 565),
 ('landing', 548),
 ('failure', 547),
 ('were', 544),
 ('taking', 518),
 ('runway', 487),
 ('it', 448),
 ('altitude', 429),
 ('cargo', 419),
 ('control', 418),
 ('miles', 417),
 ('weather', 417),
 ('attempting', 405),
 ('struck', 388),
 ('that', 360),
 ('en', 357),
 ('While', 333),
 ('ft.', 330),
 ('mountain', 328),
 ('left', 321),
 ('ground', 308),
 ('right', 301),
 ('A', 299),
 ('poor', 295),
 ('when', 294),
 ('land', 294),
 ('short', 293),
 ('due', 288),
 ('loss', 272),
 ('lo

There are certainly more sophisticated ways to do that but a quick glance reveals that the most common relevant words are: <br /> - 'engine'(700) indicating engine failure<br />  - 'approach' (607) and 'landing' (548) showing a connection to when the plane was trying to land.<br /> - 'weather' (417) indicating bad conditions<br /> - 'struck' (388) and 'mountain' 328 indicating human error while on route<br /> - 'fuel' (220)